In [1]:
import pandas as pd
import os
import numpy as np
from ast import literal_eval

In [ ]:
DATA_DIR = os.getenv("DATA_DIR")
org_data_api = os.path.join(DATA_DIR, "organisations_from_api.csv")

In [3]:
df = pd.read_csv(org_data_api)

In [4]:
df

,analytics_identifier,child_organisations,details,format,id,parent_organisations,superseded_organisations,superseding_organisations,title,updated_at,web_url
0,OT1276,[],"{'slug': 'academy-for-social-justice', 'abbrev...",Other,https://www.gov.uk/api/organisations/academy-f...,[{'id': 'https://www.gov.uk/api/organisations/...,[{'id': 'https://www.gov.uk/api/organisations/...,[],Academy for Social Justice,2019-05-15T12:12:17.000+01:00,https://www.gov.uk/government/organisations/ac...
1,OT1137,[],"{'slug': 'accelerated-access-review', 'abbrevi...",Other,https://www.gov.uk/api/organisations/accelerat...,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],Accelerated Access Review,2015-07-15T11:27:05.000+01:00,https://www.gov.uk/government/organisations/ac...
2,PB523,[],{'slug': 'administration-of-radioactive-substa...,Other,https://www.gov.uk/api/organisations/administr...,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],Administration of Radioactive Substances Advis...,2018-04-10T15:19:08.000+01:00,https://www.gov.uk/government/organisations/ad...
3,CO1188,[],"{'slug': 'administrative-court', 'abbreviation...",Court,https://www.gov.uk/api/organisations/administr...,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],Administrative Court,2016-07-25T15:16:18.000+01:00,https://www.gov.uk/government/organisations/ad...
4,CO1147,[],"{'slug': 'admiralty-court', 'abbreviation': No...",Court,https://www.gov.uk/api/organisations/admiralty...,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],Admiralty Court,2015-06-29T15:58:45.000+01:00,https://www.gov.uk/government/organisations/ad...
5,PB573,[],{'slug': 'advisory-committee-on-animal-feeding...,Advisory non-departmental public body,https://www.gov.uk/api/organisations/advisory-...,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],Advisory Committee on Animal Feedingstuffs,2014-10-15T15:36:52.000+01:00,https://www.gov.uk/government/organisations/ad...
6,PB336,[],{'slug': 'advisory-committee-on-business-appoi...,Advisory non-departmental public body,https://www.gov.uk/api/organisations/advisory-...,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],Advisory Committee on Business Appointments,2015-02-18T09:33:11.000+00:00,https://www.gov.uk/government/organisations/ad...
7,OT522,[],{'slug': 'advisory-committee-on-clinical-excel...,Advisory non-departmental public body,https://www.gov.uk/api/organisations/advisory-...,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],Advisory Committee on Clinical Excellence Awards,2014-10-15T15:36:41.000+01:00,https://www.gov.uk/government/organisations/ad...
8,PB392,[],{'slug': 'advisory-committee-on-conscientious-...,Advisory non-departmental public body,https://www.gov.uk/api/organisations/advisory-...,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],Advisory Committee on Conscientious Objectors,2014-10-15T15:36:18.000+01:00,https://www.gov.uk/government/organisations/ad...
9,PB574,[],{'slug': 'advisory-committee-on-novel-foods-an...,Advisory non-departmental public body,https://www.gov.uk/api/organisations/advisory-...,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],Advisory Committee on Novel Foods and Processes,2014-10-15T15:36:52.000+01:00,https://www.gov.uk/government/organisations/ad...


In [5]:
df['details'] = df['details'].map(literal_eval)
df['base_path'] = df['details'].map(lambda x: x['slug'])
df['content_id'] = df['details'].map(lambda x: x['content_id'])

In [6]:
for col in df.columns:
    if "organisations" in col:
        print("Evaluating",col)
        df[col] = df[col].map(literal_eval)

Evaluating child_organisations
Evaluating parent_organisations
Evaluating superseded_organisations
Evaluating superseding_organisations


In [7]:
def init_dict(tup):
    return {'src_base_path':tup['base_path'],
        'src_cid':tup['content_id'],
        'src_title':tup['title']}

In [8]:
r_list = [col for col in df.columns if "organisations" in col]
row_list = []
for i,tup in df.iterrows():
    for r,relations in [(r,tup[r]) for r in r_list]:
        if len(relations) > 0:
            for rel in relations:
                row_dict = init_dict(tup)
                row_dict['link_type'] = "has_"+r.replace("organisations","org")
                row_dict['target_base_path'] = rel['id'].split("/")[-1]
                row_list.append(row_dict)

edges = pd.DataFrame(row_list)  

In [9]:
df.iloc[0].parent_organisations

[{'id': 'https://www.gov.uk/api/organisations/ministry-of-justice',
  'web_url': 'https://www.gov.uk/government/organisations/ministry-of-justice'}]

In [10]:
edges

,link_type,src_base_path,src_cid,src_title,target_base_path
0,has_parent_org,academy-for-social-justice,b854f170-53c8-4098-bf77-e8ef42f93107,Academy for Social Justice,ministry-of-justice
1,has_superseded_org,academy-for-social-justice,b854f170-53c8-4098-bf77-e8ef42f93107,Academy for Social Justice,academy-for-social-justice-commissioning
2,has_parent_org,accelerated-access-review,a0f338c5-e94c-42f8-9c26-b9c2eb6850d3,Accelerated Access Review,department-of-health-and-social-care
3,has_parent_org,administration-of-radioactive-substances-advis...,b5bd9a64-4315-492e-9679-3e2f6799c769,Administration of Radioactive Substances Advis...,department-of-health-and-social-care
4,has_parent_org,administrative-court,b0bdfcf3-2763-4002-961e-a0b2d7825038,Administrative Court,hm-courts-and-tribunals-service
5,has_parent_org,admiralty-court,2585bb7b-6134-4176-8309-db184573a4c1,Admiralty Court,hm-courts-and-tribunals-service
6,has_parent_org,advisory-committee-on-animal-feedingstuffs,b02a8db0-6bbb-4010-9584-1a6623dc7012,Advisory Committee on Animal Feedingstuffs,food-standards-agency
7,has_parent_org,advisory-committee-on-business-appointments,6e414d7c-61af-4113-b039-435e461ddaf2,Advisory Committee on Business Appointments,cabinet-office
8,has_parent_org,advisory-committee-on-clinical-excellence-awards,aee6b528-c1f5-404c-bc69-d584e63e20a5,Advisory Committee on Clinical Excellence Awards,department-of-health-and-social-care
9,has_parent_org,advisory-committee-on-conscientious-objectors,a090e915-247a-4a63-a283-f22f948f27b3,Advisory Committee on Conscientious Objectors,ministry-of-defence


In [11]:
edges.link_type.unique()

array(['has_parent_org', 'has_superseded_org', 'has_superseding_org',
       'has_child_org'], dtype=object)

In [12]:
edges[edges.src_base_path=="ministry-of-justice"][0:5]

,link_type,src_base_path,src_cid,src_title,target_base_path
1219,has_child_org,ministry-of-justice,dcc907d6-433c-42df-9ffb-d9c68be5dc4d,Ministry of Justice,national-offender-management-service
1220,has_child_org,ministry-of-justice,dcc907d6-433c-42df-9ffb-d9c68be5dc4d,Ministry of Justice,office-of-the-public-guardian
1221,has_child_org,ministry-of-justice,dcc907d6-433c-42df-9ffb-d9c68be5dc4d,Ministry of Justice,hm-courts-and-tribunals-service
1222,has_child_org,ministry-of-justice,dcc907d6-433c-42df-9ffb-d9c68be5dc4d,Ministry of Justice,civil-justice-council
1223,has_child_org,ministry-of-justice,dcc907d6-433c-42df-9ffb-d9c68be5dc4d,Ministry of Justice,law-commission


### Map back target title and content_id

In [13]:
bp_cid = dict(zip(edges.src_base_path, edges.src_cid))
bp_tit = dict(zip(edges.src_base_path, edges.src_title))

In [14]:
edges['target_cid'] = edges['target_base_path'].map(bp_cid)
edges['target_title'] = edges['target_base_path'].map(bp_tit)

In [15]:
edges.columns

Index(['link_type', 'src_base_path', 'src_cid', 'src_title',
       'target_base_path', 'target_cid', 'target_title'],
      dtype='object')

In [16]:
edges = edges[['src_base_path', 'src_cid', 'src_title',
        'target_base_path', 'target_cid', 'target_title', 'link_type']]

In [17]:
edges

,src_base_path,src_cid,src_title,target_base_path,target_cid,target_title,link_type
0,academy-for-social-justice,b854f170-53c8-4098-bf77-e8ef42f93107,Academy for Social Justice,ministry-of-justice,dcc907d6-433c-42df-9ffb-d9c68be5dc4d,Ministry of Justice,has_parent_org
1,academy-for-social-justice,b854f170-53c8-4098-bf77-e8ef42f93107,Academy for Social Justice,academy-for-social-justice-commissioning,ce357bdb-6396-426a-9f1f-8cbfb444cffd,Academy for Social Justice Commissioning,has_superseded_org
2,accelerated-access-review,a0f338c5-e94c-42f8-9c26-b9c2eb6850d3,Accelerated Access Review,department-of-health-and-social-care,7cd6bf12-bbe9-4118-8523-f927b0442156,Department of Health and Social Care,has_parent_org
3,administration-of-radioactive-substances-advis...,b5bd9a64-4315-492e-9679-3e2f6799c769,Administration of Radioactive Substances Advis...,department-of-health-and-social-care,7cd6bf12-bbe9-4118-8523-f927b0442156,Department of Health and Social Care,has_parent_org
4,administrative-court,b0bdfcf3-2763-4002-961e-a0b2d7825038,Administrative Court,hm-courts-and-tribunals-service,6f757605-ab8f-4b62-84e4-99f79cf085c2,HM Courts & Tribunals Service,has_parent_org
5,admiralty-court,2585bb7b-6134-4176-8309-db184573a4c1,Admiralty Court,hm-courts-and-tribunals-service,6f757605-ab8f-4b62-84e4-99f79cf085c2,HM Courts & Tribunals Service,has_parent_org
6,advisory-committee-on-animal-feedingstuffs,b02a8db0-6bbb-4010-9584-1a6623dc7012,Advisory Committee on Animal Feedingstuffs,food-standards-agency,447f8075-1c8c-40d3-8119-3e45ea9e6496,Food Standards Agency,has_parent_org
7,advisory-committee-on-business-appointments,6e414d7c-61af-4113-b039-435e461ddaf2,Advisory Committee on Business Appointments,cabinet-office,96ae61d6-c2a1-48cb-8e67-da9d105ae381,Cabinet Office,has_parent_org
8,advisory-committee-on-clinical-excellence-awards,aee6b528-c1f5-404c-bc69-d584e63e20a5,Advisory Committee on Clinical Excellence Awards,department-of-health-and-social-care,7cd6bf12-bbe9-4118-8523-f927b0442156,Department of Health and Social Care,has_parent_org
9,advisory-committee-on-conscientious-objectors,a090e915-247a-4a63-a283-f22f948f27b3,Advisory Committee on Conscientious Objectors,ministry-of-defence,d994e55c-48c9-4795-b872-58d8ec98af12,Ministry of Defence,has_parent_org


### Add org_id to aid in creating node list

In [18]:
org_dict = dict(zip(df.title, df.analytics_identifier))

In [19]:
edges['src_orgid']= edges['src_title'].map(org_dict)
edges['target_orgid']= edges['target_title'].map(org_dict)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
edges

,src_base_path,src_cid,src_title,target_base_path,target_cid,target_title,link_type,src_orgid,target_orgid
0,academy-for-social-justice,b854f170-53c8-4098-bf77-e8ef42f93107,Academy for Social Justice,ministry-of-justice,dcc907d6-433c-42df-9ffb-d9c68be5dc4d,Ministry of Justice,has_parent_org,OT1276,D18
1,academy-for-social-justice,b854f170-53c8-4098-bf77-e8ef42f93107,Academy for Social Justice,academy-for-social-justice-commissioning,ce357bdb-6396-426a-9f1f-8cbfb444cffd,Academy for Social Justice Commissioning,has_superseded_org,OT1276,OT1208
2,accelerated-access-review,a0f338c5-e94c-42f8-9c26-b9c2eb6850d3,Accelerated Access Review,department-of-health-and-social-care,7cd6bf12-bbe9-4118-8523-f927b0442156,Department of Health and Social Care,has_parent_org,OT1137,D12
3,administration-of-radioactive-substances-advis...,b5bd9a64-4315-492e-9679-3e2f6799c769,Administration of Radioactive Substances Advis...,department-of-health-and-social-care,7cd6bf12-bbe9-4118-8523-f927b0442156,Department of Health and Social Care,has_parent_org,PB523,D12
4,administrative-court,b0bdfcf3-2763-4002-961e-a0b2d7825038,Administrative Court,hm-courts-and-tribunals-service,6f757605-ab8f-4b62-84e4-99f79cf085c2,HM Courts & Tribunals Service,has_parent_org,CO1188,EA73
5,admiralty-court,2585bb7b-6134-4176-8309-db184573a4c1,Admiralty Court,hm-courts-and-tribunals-service,6f757605-ab8f-4b62-84e4-99f79cf085c2,HM Courts & Tribunals Service,has_parent_org,CO1147,EA73
6,advisory-committee-on-animal-feedingstuffs,b02a8db0-6bbb-4010-9584-1a6623dc7012,Advisory Committee on Animal Feedingstuffs,food-standards-agency,447f8075-1c8c-40d3-8119-3e45ea9e6496,Food Standards Agency,has_parent_org,PB573,D102
7,advisory-committee-on-business-appointments,6e414d7c-61af-4113-b039-435e461ddaf2,Advisory Committee on Business Appointments,cabinet-office,96ae61d6-c2a1-48cb-8e67-da9d105ae381,Cabinet Office,has_parent_org,PB336,D2
8,advisory-committee-on-clinical-excellence-awards,aee6b528-c1f5-404c-bc69-d584e63e20a5,Advisory Committee on Clinical Excellence Awards,department-of-health-and-social-care,7cd6bf12-bbe9-4118-8523-f927b0442156,Department of Health and Social Care,has_parent_org,OT522,D12
9,advisory-committee-on-conscientious-objectors,a090e915-247a-4a63-a283-f22f948f27b3,Advisory Committee on Conscientious Objectors,ministry-of-defence,d994e55c-48c9-4795-b872-58d8ec98af12,Ministry of Defence,has_parent_org,PB392,D17


### Save

In [24]:
edges.to_csv(os.path.join(DATA_DIR, "org_edgelist_from_api.csv"), index=False)